In [ ]:
#select all samples which contain the keyword phage
#/mnt/mnemo4/microbeatlas/mapdata/20210104map1/samples.info.keywords 




In [1]:


#inp="/mnt/gaia/jfmrod/gaia/usr/bin/mhpc-clust -t samples -nthreads 30 -dfunc braycurtis -makecluststats /mnt/mnemo4/microbeatlas/mapdata/20200307/otutables/samples-otus.90.mapped.gz /mnt/mnemo6/lukas/Python/all_otu_samplelist/all_99_otus_samples_v0.2.2_.csv /mnt/mnemo6/lukas/Python/evo_treedists/Chloroflexi/Chloroflexi_pairs.tbl >>/mnt/mnemo6/lukas/Python/evo_treedists/Chloroflexi/Chloroflexi_BC_2.tbl"

#inp="/mnt/gaia/jfmrod/gaia/usr/bin/mhpc-clust -t samples -nthreads 30 -dfunc braycurtis -makecluststats /mnt/mnemo4/microbeatlas/mapdata/20200307/otutables/samples-otus.90.mapped.gz /mnt/mnemo6/lukas/Python/all_otu_samplelist/all_99_otus_samples_v0.2.2_.csv /mnt/mnemo6/lukas/Python/evo_treedists/"+taxname+"/"+taxname+"_pairs.tbl >>/mnt/mnemo6/lukas/Python/evo_treedists/"+taxname+"/"+taxname+"_BC.tbl"


grep phage /mnt/mnemo4/microbeatlas/mapdata/20210104map1/samples.info.keywords 


example file: /mnt/mnemo7/jfmrod/databases.mnemo4/databases/ncbi/download/sra/sra-instant/reads/ByRun/sra/SRR/SRR975/SRR975391/SRR975391.fa.gz

In [1]:

import h5py
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix

def read_mapdb_h5(h5_path, otu_data=True, meta_data=True):
    """Example file: /mnt/mnemo3/janko/data/microbe_atlas/sruns-otus.97.otutable_plusMetaNoHeader_taxonomy_unmapped.h5"""
    f = h5py.File(h5_path, 'r')
    result_dict = {}

    # read otu data
    if otu_data:
        data_handle = f["otu_table"]
        col_ptr, nzvals, rowindices, otu_index, sample_index = [np.array(data_handle[sub_key]) for sub_key in ['data_colptr', 'data_nzval', 'data_rowval', 'oids', 'sids']]

        ## correct indexing (julia starts at 1)
        col_ptr -= 1
        rowindices -= 1

        otutable_sparse = csc_matrix((nzvals,rowindices,col_ptr), shape=(data_handle["m"][()],data_handle["n"][()]))
        result_dict["otu_data"] = {"otu_table": otutable_sparse, "otu_index": otu_index, "sample_index": sample_index}

    # read meta data
    if meta_data:
        meta_handle = f["meta_data"]
        result_dict["meta_data"] = {sub_key: pd.DataFrame(np.array(meta_handle[sub_key]).T) for sub_key in meta_handle.keys()}

    f.close()

    return result_dict



In [2]:
#select OTU level here: in This case 99%
f2=read_mapdb_h5("/mnt/mnemo3/janko/data/microbe_atlas/hdf5/v0.2.2/samples-otus.97.mapped.remap.h5")

keyw=f2["meta_data"]["keywords"]

readspec=f2["meta_data"]["readspec"]

#checking all the samples which are both wgs as well as copntain phage in their keyword
wgs_ind=keyw[keyw[3]=="WGS"].index


wgs=keyw[keyw[3]=="WGS"]

In [3]:
f2["otu_data"]["otu_table"]

<2174192x137800 sparse matrix of type '<class 'numpy.int32'>'
	with 645021862 stored elements in Compressed Sparse Column format>

In [38]:
all_runs_with_phages=wgs[wgs[1].str.contains("animal")][0]

In [ ]:
#maybe do this faster by first checking something like which samples are WGS and then check if sample in WGS: etc 

zcat PATH/file.gz >> TMPPATH/file.fa

In [ ]:
#zcat to copy all files into a tmp location

In [ ]:
#get all wgs path
for i in wgs[0]:
    inp= "grep -m 1 "+i.split(".")[0]+" /home/jfmrod/mnemo4/work/mapdb/data/all.runs.fa.map >> /mnt/mnemo6/lukas/Python/ribozymes/version4/wgs_sample_paths.txt"

    subprocess.run(inp, shell=True)

In [12]:
import subprocess
f=open("gut_sample_paths.txt")
f_lin=f.readlines()

for i in range(len(f_lin)):
    f_lin[i]=f_lin[i].strip()

#get the total length_ split it into 20 parts?
length_total=len(f_lin)

length_part=int(length_total/60)

#global_fasta=""
def open_fils(filename,number):
    #where to save the found sequence locations, to later grep out the fasta stuff
    #or do directly in bash via grep, whats better?
    #global global_fasta
    #locations=[]
    
    filepath="/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/rnarobo_results/"+str(number)+"/"+filename+"_rnarobo_result.txt"
    filepath_fasta="/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/"+str(number)+"/"+filename+".fa"
    f1=open(filepath)
    f_l1=f1.readlines()
    f1.close()
    #going thorugh the results in one file: check for start of found seq ("  "), then append the start position, end position, sequencename (important!) and sequence itself
    #counter=0
    #start=[]
    #end=[]
    seqnam=[]
    #sequence=[]
    #go trhough each line of rnarobo output
    for line in f_l1:
        if line[0:2]=="  ":
            line=line.strip().split()
            #start.append(line[0])
            #end.append(line[1])
            seqnam.append(line[2])
            counter=1
        #add the sequence of the ribozyme    
        #elif counter==1:
            #line=line.strip()
            #sequence.append(line)
            #counter=0
    #all reads where a ribozyme was found        
    unique_seqnam=set(seqnam)
    print(unique_seqnam)
    #look for all the 
    sequence_saving=""
    for seqnam in unique_seqnam:
        inp= "sed -n '/"+seqnam+" /,/>/p' "+filepath_fasta
        l=subprocess.run(inp,shell=True,capture_output=True,text=True)
        sequence_saving+=l.stdout
        #global_fasta+=l.stdout
    f2=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/rnarobo_results_seqs/"+str(number)+"/"+filename+"_seqs_rnarobo_result.txt","w")
    f2.write(sequence_saving)
    f2.close
        #print(l.stdout)

In [21]:
#Trial to figure out how we can save the sequence in question without the wrong header in the end
import subprocess
sequence_saving=""

inp= "sed -n '/D7ZQJ5M1:705:C5GNWACXX:5:1101:4750:2265 /,/>/p' /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/1/SRR2082149.fa"
l=subprocess.run(inp,shell=True,capture_output=True,text=True)
sequence_saving+=l.stdout

">".join(sequence_saving.split(">")[:-1])

''

In [24]:
l

CompletedProcess(args="sed -n '/D7ZQJ5M1:705:C5GNWACXX:5:1101:4750:2265 /,/>/p' /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/1/SRR2082149.fa", returncode=2, stdout='', stderr="sed: can't read /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/1/SRR2082149.fa: No such file or directory\n")

In [42]:
for number in range(3,21):
    
    stri="""
import subprocess
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/rnarobo_results/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/rnarobo_results_seqs/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
f=open("gut_sample_paths.txt")
f_lin=f.readlines()

for i in range(len(f_lin)):
    f_lin[i]=f_lin[i].strip()

#get the total length_ split it into 20 parts?
length_total=len(f_lin)

length_part=int(length_total/60)

#global_fasta=""
def open_fils(filename):
    #where to save the found sequence locations, to later grep out the fasta stuff
    #or do directly in bash via grep, whats better?
    #global global_fasta
    #locations=[]

    filepath="/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/rnarobo_results/"+str("""+str(number)+""")+"/"+filename+"_rnarobo_result.txt"
    filepath_fasta="/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/"+str("""+str(number)+""")+"/"+filename+".fa"
    f1=open(filepath)
    f_l1=f1.readlines()
    f1.close()
    #going thorugh the results in one file: check for start of found seq ("  "), then append the start position, end position, sequencename (important!) and sequence itself
    #counter=0
    #start=[]
    #end=[]
    seqnam=[]
    #sequence=[]
    #go trhough each line of rnarobo output
    for line in f_l1:
        if line[0:2]=="  ":
            line=line.strip().split()
            #start.append(line[0])
            #end.append(line[1])
            seqnam.append(line[2])
            counter=1
        #add the sequence of the ribozyme    
        #elif counter==1:
            #line=line.strip()
            #sequence.append(line)
            #counter=0
    #all reads where a ribozyme was found        
    unique_seqnam=set(seqnam)
    print(unique_seqnam)
    #look for all the 
    sequence_saving=""
    for seqnam in unique_seqnam:
        inp= "sed -n '/"+seqnam+" /,/>/p' "+filepath_fasta
        l=subprocess.run(inp,shell=True,capture_output=True,text=True)
        sequence_saving+=l.stdout
        #global_fasta+=l.stdout
    sequence_saving=">".join(sequence_saving.split(">")[:-1])
    f2=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/rnarobo_results_seqs/"+str("""+str(number)+""")+"/"+filename+"_seqs_rnarobo_result.txt","w")
    f2.write(sequence_saving)
    f2.close
    #print(l.stdout)
for path in f_lin[length_part*"""+str(number)+""":length_part*("""+str(number)+"""+1)]:

    #extract the file to a new location, unzip it
    name=path.split("/")[-1].split(".")[0]
    ending=path.split("/")[-1].split(".")[-1]
    if ending=="gz":
        inp= "zcat "+path+" >> /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/"+str("""+str(number)+""")+"/"+name+".fa"
    elif ending=="bz2":
        #dc to write to stdout and keep original file
        inp= "bzip2 -dc "+path+">> /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/"+str("""+str(number)+""")+"/"+name+".fa"
    else:
        inp=""
    #run the decompression, depoedning on which ending we have    
    subprocess.run(inp, shell=True)

    #run rnarobo, with --nratio do exclude the bad quality baseppairs
    inp2= "/mnt/mnemo6/lukas/anaconda3/bin/rnarobo-2.1.0-linux64 -c --nratio 0.1 /mnt/mnemo6/lukas/Python/ribozymes/version3/motif2_precise.txt /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/"+str("""+str(number)+""")+"/"+name+".fa >> /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/rnarobo_results/"+str("""+str(number)+""")+"/"+name+"_rnarobo_result.txt"
    subprocess.run(inp2, shell=True)

    #cutting out sequence and saving that, as well as adding it to global variable global_fasta, which then contains all potential seuqences to scan with trnascan
    open_fils(name)

    inp3= "rm /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3/"+str("""+str(number)+""")+"/"+name+".fa"
    subprocess.run(inp3, shell=True)
    #sed through the results and cut out interesting parts, save to file? then run trnascan, or in the end?
"""

    f=open("test"+str(number)+".py","w")
    f.write(stri)
    f.close()


In [1]:
#we are replacing the original script and insetad saving it into v3_t- plus, afterwards we are also including the trna scan here
stringa='''for number in range(1,21):
    
    stri="""
import subprocess
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/rnarobo_results/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/rnarobo_results_seqs/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/trnascan/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
f=open("gut_sample_paths.txt")
f_lin=f.readlines()

for i in range(len(f_lin)):
    f_lin[i]=f_lin[i].strip()

#get the total length_ split it into 20 parts?
length_total=len(f_lin)

length_part=int(length_total/60)

#global_fasta=""
def open_fils(filename):
    #where to save the found sequence locations, to later grep out the fasta stuff
    #or do directly in bash via grep, whats better?
    #global global_fasta
    #locations=[]

    filepath="/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/rnarobo_results/"+str("""+str(number)+""")+"/"+filename+"_rnarobo_result.txt"
    filepath_fasta="/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/"+str("""+str(number)+""")+"/"+filename+".fa"
    f1=open(filepath)
    f_l1=f1.readlines()
    f1.close()
    #going thorugh the results in one file: check for start of found seq ("  "), then append the start position, end position, sequencename (important!) and sequence itself
    #counter=0
    #start=[]
    #end=[]
    seqnam=[]
    #sequence=[]
    #go trhough each line of rnarobo output
    for line in f_l1:
        if line[0:2]=="  ":
            line=line.strip().split()
            #start.append(line[0])
            #end.append(line[1])
            seqnam.append(line[2])
            counter=1
        #add the sequence of the ribozyme    
        #elif counter==1:
            #line=line.strip()
            #sequence.append(line)
            #counter=0
    #all reads where a ribozyme was found        
    unique_seqnam=set(seqnam)
    print(unique_seqnam)
    #look for all the 
    sequence_saving=""
    for seqnam in unique_seqnam:
        inp= "sed -n '/"+seqnam+" /,/>/p' "+filepath_fasta
        l=subprocess.run(inp,shell=True,capture_output=True,text=True)
        if len(l.stdout.split(">")) > 2:
            sequence_saving+=">".join(l.stdout.split(">")[:-1])
                    #print(seqnam)
        else:
            sequence_saving+=l.stdout
    f2=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/rnarobo_results_seqs/"+str("""+str(number)+""")+"/"+filename+"_seqs_rnarobo_result.txt","w")

    #sequence_saving2=">".join(sequence_saving.split(">")[:-1])
    f2.write(sequence_saving)
    f2.close
    #print(l.stdout)
for path in f_lin[length_part*"""+str(number)+""":length_part*("""+str(number)+"""+1)]:

    #extract the file to a new location, unzip it
    name=path.split("/")[-1].split(".")[0]
    ending=path.split("/")[-1].split(".")[-1]
    if ending=="gz":
        inp= "zcat "+path+" >> /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/"+str("""+str(number)+""")+"/"+name+".fa"
    elif ending=="bz2":
        #dc to write to stdout and keep original file
        inp= "bzip2 -dc "+path+">> /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/"+str("""+str(number)+""")+"/"+name+".fa"
    else:
        inp=""
    #run the decompression, depoedning on which ending we have    
    subprocess.run(inp, shell=True)

    #run rnarobo, with --nratio do exclude the bad quality baseppairs
    inp2= "/mnt/mnemo6/lukas/anaconda3/bin/rnarobo-2.1.0-linux64 -c --nratio 0.1 /mnt/mnemo6/lukas/Python/ribozymes/version3/motif2_precise_T.txt /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/"+str("""+str(number)+""")+"/"+name+".fa >> /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/rnarobo_results/"+str("""+str(number)+""")+"/"+name+"_rnarobo_result.txt"
    subprocess.run(inp2, shell=True)

    #cutting out sequence and saving that, as well as adding it to global variable global_fasta, which then contains all potential seuqences to scan with trnascan
    open_fils(name)

    inp3= "rm /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/"+str("""+str(number)+""")+"/"+name+".fa"
    subprocess.run(inp3, shell=True)
    #sed through the results and cut out interesting parts, save to file? then run trnascan, or in the end?
    inp4="/mnt/mnemo6/lukas/anaconda3/envs/sar3/bin/tRNAscan-SE -G --detail /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/rnarobo_results_seqs/"+str("""+str(number)+""")+"/"+name+"_seqs_rnarobo_result.txt -o /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/trnascan/"+str("""+str(number)+""")+"/"+name+"trna_result.txt -f /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v3_T/trnascan/"+str("""+str(number)+""")+"/"+name+"trna_secondary.tbl"
    subprocess.run(inp4, shell=True)
"""

    f=open("test"+str(number)+".py","w")
    f.write(stri)
    f.close()
'''

print(stringa.replace("v3_T","v4"))

for number in range(1,21):
    
    stri="""
import subprocess
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results_seqs/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/trnascan/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
f=open("gut_sample_paths.txt")
f_lin=f.readlines()

for i in range(len(f_lin)):
    f_lin[i]=f_lin[i].strip()

#get the total length_ split it into 20 parts?
length_total=len(f_lin)

length_part=int(length_total/60)

#global_fasta=""
def open_fils(filename):
    #where to save the found sequence locations, to later grep out the fasta stuff
    #or do di

In [3]:
for number in range(1,61):
    
    stri="""
import subprocess
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results_seqs/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
inp_start="mkdir /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/trnascan/"+str("""+str(number)+""")
subprocess.run(inp_start, shell=True)
f=open("wgs_sample_paths.txt")
f_lin=f.readlines()

for i in range(len(f_lin)):
    f_lin[i]=f_lin[i].strip()

#get the total length_ split it into 60 parts?
length_total=len(f_lin)

length_part=int(length_total/60)

#global_fasta=""
def open_fils(filename):
    #where to save the found sequence locations, to later grep out the fasta stuff
    #or do directly in bash via grep, whats better?
    #global global_fasta
    #locations=[]

    filepath="/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results/"+str("""+str(number)+""")+"/"+filename+"_rnarobo_result.txt"
    filepath_fasta="/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/"+str("""+str(number)+""")+"/"+filename+".fa"
    f1=open(filepath)
    f_l1=f1.readlines()
    f1.close()
    #going thorugh the results in one file: check for start of found seq ("  "), then append the start position, end position, sequencename (important!) and sequence itself
    #counter=0
    #start=[]
    #end=[]
    seqnam=[]
    #sequence=[]
    #go trhough each line of rnarobo output
    for line in f_l1:
        if line[0:2]=="  ":
            line=line.strip().split()
            #start.append(line[0])
            #end.append(line[1])
            seqnam.append(line[2])
            counter=1
        #add the sequence of the ribozyme    
        #elif counter==1:
            #line=line.strip()
            #sequence.append(line)
            #counter=0
    #all reads where a ribozyme was found        
    unique_seqnam=set(seqnam)
    print(unique_seqnam)
    #look for all the 
    sequence_saving=""
    for seqnam in unique_seqnam:
        inp= "sed -n '/"+seqnam+" /,/>/p' "+filepath_fasta
        l=subprocess.run(inp,shell=True,capture_output=True,text=True)
        if len(l.stdout.split(">")) > 2:
            sequence_saving+=">".join(l.stdout.split(">")[:-1])
                    #print(seqnam)
        else:
            sequence_saving+=l.stdout
    f2=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results_seqs/"+str("""+str(number)+""")+"/"+filename+"_seqs_rnarobo_result.txt","w")

    #sequence_saving2=">".join(sequence_saving.split(">")[:-1])
    f2.write(sequence_saving)
    f2.close
    #print(l.stdout)
for path in f_lin[length_part*"""+str(number)+""":length_part*("""+str(number)+"""+1)]:

    #extract the file to a new location, unzip it
    name=path.split("/")[-1].split(".")[0]
    ending=path.split("/")[-1].split(".")[-1]
    if ending=="gz":
        inp= "zcat "+path+" >> /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/"+str("""+str(number)+""")+"/"+name+".fa"
    elif ending=="bz2":
        #dc to write to stdout and keep original file
        inp= "bzip2 -dc "+path+">> /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/"+str("""+str(number)+""")+"/"+name+".fa"
    else:
        inp=""
    #run the decompression, depoedning on which ending we have    
    subprocess.run(inp, shell=True)

    #run rnarobo, with --nratio do exclude the bad quality baseppairs
    inp2= "/mnt/mnemo6/lukas/anaconda3/bin/rnarobo-2.1.0-linux64 -c --nratio 0.1 /mnt/mnemo6/lukas/Python/ribozymes/version4/motif2_precise.txt /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/"+str("""+str(number)+""")+"/"+name+".fa >> /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results/"+str("""+str(number)+""")+"/"+name+"_rnarobo_result.txt"
    subprocess.run(inp2, shell=True)

    #cutting out sequence and saving that, as well as adding it to global variable global_fasta, which then contains all potential seuqences to scan with trnascan
    open_fils(name)

    inp3= "rm /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/"+str("""+str(number)+""")+"/"+name+".fa"
    subprocess.run(inp3, shell=True)
    #sed through the results and cut out interesting parts, save to file? then run trnascan, or in the end?
    inp4="/mnt/mnemo6/lukas/anaconda3/envs/sar3/bin/tRNAscan-SE -G --detail /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/rnarobo_results_seqs/"+str("""+str(number)+""")+"/"+name+"_seqs_rnarobo_result.txt -o /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/trnascan/"+str("""+str(number)+""")+"/"+name+"trna_result.txt -f /mnt/mnemo6/lukas/Python/ribozymes/phage_samples/v4/trnascan/"+str("""+str(number)+""")+"/"+name+"trna_secondary.tbl"
    subprocess.run(inp4, shell=True)
"""

    f=open("test"+str(number)+".py","w")
    f.write(stri)
    f.close()



In [ ]:

"""
com_fas=global_fasta.split("\n")

fasta_corr=[]
checker=0
fasta_part=""
#to -1 because last one is empty for whatever reason
for line in com_fas[:-1]:
    
    if line[0]==">":
        
 
        if checker==0:
            fasta_corr.append(fasta_part)
            fasta_part=""
            line=line.split()
            fasta_corr.append(line[0])
            checker=1
        else:
            checker=0
    else:
        fasta_part+=line.strip()
        

fasta_corr=fasta_corr[1:]

fast_seq="\n".join(fasta_corr)

fast_seq=fast_seq.replace(".","_")

f= open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/new/trna_scan/all_sequences.fasta","w+")
f.write(fast_seq)
f.close()
#trnascan here
"""

In [36]:
global_fasta

''

In [42]:
rs_1=open("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/new/trna_scan/trnascan_results.txt")
rs_s=rs_1.readlines()
rs_1.close()

In [90]:
#create dict for trna beginnings
dicti={}
for line in rs_s[4:]:
    #create a dictionary with seq name as key and the starting position as value
        line=line.strip().split()
        if line[0] in dicti:
            dicti[line[0]].append(int(line[3]))
        else:
            dicti[line[0]]=[int(line[3])]
            
        

In [59]:
#lists all the files in the directory
filenames_filt=[]
filenames=os.listdir("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/new/rnarobo_results/")
for fi in filenames:
    if fi[-3:]=="txt":
        filenames_filt.append(fi)

In [69]:
#open multiple results files and saving to dict
dicti_r={}
import os
for filename in filenames_filt:
   with open(os.path.join("/mnt/mnemo6/lukas/Python/ribozymes/phage_samples/new/rnarobo_results/", filename), 'r') as f: # open in readonly mode
      # do your stuff
        f_l=f.readlines()
        #means at least one ribo was found
        if len(f_l)>17:
            for linea in f_l[11:-6]:
                if linea[0]!="|":
                    line=linea.replace(".","_")
                    line=line.strip().split()
                    if line[2] in dicti_r:
                        dicti_r[line[2]].append(int(line[0]))
                    else:
                        dicti_r[line[2]]=[int(line[0])]

            


In [97]:
print(sum([len(dicti_r[x]) for x in dicti_r if isinstance(dicti_r[x], list)]))

7533


In [71]:
matching=[]
for key in dicti_r:
    if key in dicti:
        for value in dicti_r[key]:
            for value2 in dicti[key]:
                if value-value2 in range(-15,15):
                    matching.append([key,value,value2])

In [98]:
matching

[['SRR5088933_4044330', 189, 177],
 ['SRR6379761_193842', 69, 70],
 ['SRR6379761_839598', 69, 67],
 ['SRR6379761_6054261', 229, 231],
 ['SRR6379761_7115974', 218, 219],
 ['SRR6379761_7990996', 69, 67],
 ['SRR6379761_9807507', 244, 243],
 ['SRR6379761_16636702', 205, 206],
 ['SRR6379761_17261682', 223, 231],
 ['SRR6379761_19449059', 223, 231],
 ['SRR6379761_20325740', 223, 231],
 ['SRR6379761_20502096', 75, 80],
 ['SRR6379761_21551329', 214, 212],
 ['SRR6379761_21880813', 214, 212],
 ['SRR6379761_23140049', 218, 219],
 ['SRR6379761_25657473', 69, 67],
 ['SRR6379761_29330552', 69, 67],
 ['SRR6379761_33382391', 69, 67]]

In [74]:
import pandas as pd

In [79]:
#blast result

blstr=pd.read_csv("/mnt/mnemo6/lukas/Python/ribozymes/trna_database/test_3.tbl",sep="\t",header=None)

In [84]:
blstr.columns=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart" ,"qend", "sstart" ,"send", "evalue", "bitscore"]

In [91]:
dicti_t_2={}
for ids in range(len(blstr)):
    #create a dictionary with seq name as key and the starting position as value
        if blstr.iloc[ids,0] in dicti_t_2:
            dicti_t_2[blstr.iloc[ids,0]].append(int(blstr.iloc[ids,7]))
        else:
            dicti_t_2[blstr.iloc[ids,0]]=[int(blstr.iloc[ids,7])]

In [93]:
matching_2=[]
for key in dicti_r:
    if key in dicti_t_2:
        for value in dicti_r[key]:
            for value2 in dicti_t_2[key]:
                if value-value2 in range(-15,15):
                    matching_2.append([key,value,value2])

In [94]:
matching_2

[['SRR5088933_847575', 142, 139],
 ['SRR5088933_847575', 142, 139],
 ['SRR5088933_2776739', 144, 141],
 ['SRR5088933_2776739', 144, 141],
 ['SRR5088933_3167187', 45, 42],
 ['SRR5088933_3167187', 45, 42],
 ['SRR5088933_3817122', 138, 135],
 ['SRR5088933_3817122', 138, 135],
 ['SRR5088933_4004101', 34, 31],
 ['SRR5088933_4004101', 34, 31],
 ['SRR5088933_5160924', 38, 35],
 ['SRR5088933_5160924', 38, 35],
 ['SRR5088933_7329507', 143, 140],
 ['SRR5088933_7329507', 143, 140],
 ['SRR5088933_7743988', 147, 144],
 ['SRR5088933_7743988', 147, 144],
 ['SRR5088933_7757114', 139, 136],
 ['SRR5088933_7757114', 139, 136],
 ['SRR5088933_7769075', 48, 45],
 ['SRR5088933_7769075', 48, 45],
 ['SRR5088933_8052701', 34, 31],
 ['SRR5088933_8052701', 34, 31],
 ['SRR5088933_8145875', 40, 37],
 ['SRR5088933_8145875', 40, 37],
 ['SRR5088933_8166443', 145, 142],
 ['SRR5088933_8166443', 145, 142],
 ['SRR5088933_8191347', 42, 39],
 ['SRR5088933_8191347', 42, 39],
 ['SRR5088933_8313991', 32, 29],
 ['SRR5088933_831

In [ ]:
#maybe parellelizing? 
import subprocess

template = 'python script.py {} {} {} {}'

args = [[1, 2, 3, 4], [5, 6, 7, 8]]

# Run commands in parallel
processes = []

for arg in args:
    command = template.format(*[str(a) for a in arg])
    process = subprocess.Popen(command, shell=True)
    processes.append(process)

# Collect statuses
output = [p.wait() for p in processes]